In [1]:
%matplotlib inline

## Automatic differentiation with ``torch.autograd``

신경망을 훈련할 때 가장 자주 사용되는 알고리즘은 **back propragation(역전파)** 입니다. 이 알고리즘에서 파라미터(모델의 가중치)는 주어진 파라미터에 대한 손실 함수(loss function)의 **gradient(기울기)** 에 따라 조정됩니다.

이러한 기울기를 계산하기 위해서는 PyTorch에서는 `torch.autograd`라는 미분 엔진을 사용하며, 모든 계산 그래프에 대해 기울기 자동 계산을 지원합니다.

Input `x`, 매개 변수 `w` 및 `b`인 단순한 One-Layer 신경망을 가정하면, PyTorch에서는 아래와 같이 정의할 수 있습니다.

In [2]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

## Tensors, Functions and Computational graph

아래 코드는 다음의 **계산 그래프(Computational Graph)**를 정의합니다.

![Loss의 기울기를 계산하기 위해 두 매개 변수 `w`와 `b`가 있는 계산 그래프를 보여주는 다이어그램](./images/computational-graph.png)

이 네트워크에서 `w`와 `b`는 최적화가 필요한 **파라미터**로, 이러한 변수에 대한 손실 함수(Loss Function)의 기울기를 계산할 수 있어야 합니다. 이를 위해 해당 Tensor의 `requires_grad`를 설정합니다.

> **Note:** Tensor 생성 시 `requires_grad` 값을 설정하거나 나중에 `x.requires_grad_(True)` 메소드를 사용하여 설정할 수 있습니다.

계산 그래프를 구성하기 위해 Tensor에 적용하는 함수는 `Function` 클래스의 객체로, 이 객체는 *forward* 방향으로 함수를 계산하는 방법과 *backward propagation* 단계에서 미분을 계산하는 방법이 내장되어 있습니다. 역전파 함수에 대한 참조는 Tensor의 `grad_fn`에 저장되며, [문서](https://pytorch.org/docs/stable/autograd.html#function)에서 자세한 정보를 찾을 수 있습니다.

In [3]:
print('Gradient function for z =',z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x00000280CC630CA0>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward object at 0x00000280CC630310>


## Computing gradients

신경망에서 파라미터의 가중치를 최적화하려면 파라미터에 대한 손실 함수의 도함수, 즉 $\frac{\partial loss}{\partial w}$, $\frac{\partial loss}{\partial b}$가 필요하며, 이를 위해 `loss.backward()`를 호출한 다음, `w.grad`와 `b.grad`의 값을 계산합니다.

In [4]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.2739, 0.0490, 0.3279],
        [0.2739, 0.0490, 0.3279],
        [0.2739, 0.0490, 0.3279],
        [0.2739, 0.0490, 0.3279],
        [0.2739, 0.0490, 0.3279]])
tensor([0.2739, 0.0490, 0.3279])


> **Note:** `requires_grad`가 ``True``로 설정된 계산 그래프의 리프 노드(leaf nodes)에 대한 `grad` 속성만 가져올 수 있습니다. 그래프의 다른 모든 노드에 대해서는 기울기를 사용할 수 없으며, 성능 상의 이유로 주어진 그래프에서 ``backward``를 한 번만 사용하여 기울기 계산을 수행할 수 있습니다. 동일한 그래프에서 ``backward``를 여러 번 수행하여야 하는 경우, ``retain_graph=True``로 설정하면 됩니다.


## Disabling gradient tracking

기본적으로 `requires_grad=True`인 모든 Tensor는 계산 기록을 추적(tracking)하고 기울기 계산을 지원합니다. 그러나 이를 수행할 필요가 없는 경우가 존재합니다. 예를 들어 모델을 학습하고 일부 입력 데이터에 적용하려는 경우, 즉 네트워크를 **순방향(forward)**으로만 수행하려는 경우가 있습니다. 이때는 `torch.no_grad()`를 block하여 계산을 진행하지 않을 수 있습니다.

In [5]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


동일한 결과를 얻는 또 다른 방법은 Tensor에서 ``detach()`` 메소드를 사용하는 것입니다.

In [6]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


기울기 추적을 비활성화하려는 이유는 다음과 같습니다.

  - 신경망의 일부 매개 변수를 고정하고자 할 때(Fine-Tuning) 사용합니다. (참조 : [fine tuning a pre-trained network](https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html))
  - 기울기를 추적하지 않는 Tensor에 대한 계산이 더 효율적이기 때문에 순방향으로만 수행하려는 경우 계산 속도가 향상될 수 있습니다.

More on Computational Graphs
----------------------------
개념적으로, Autograd는 [Function](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function) 객체로 구성된 directed acyclic graph (DAG)에 데이터(Tensors) 및 모든 실행 연산(결과적인 새 텐서와 함께)의 레코드를 보관한다.

이 DAG에서 리프는 입력 Tensors, 루트는 출력 Tensors입니다. 이 그래프를 루트에서 리프까지 추적하면 체인 룰을 사용하여 기울기를 자동으로 계산할 수 있습니다.

정방향 패스에서 autograd는 두 가지 작업을 동시에 수행합니다:

- 요청한 작업을 실행하여 Tensors를 계산합니다.
- DAG에서 작업의 *gradient function*을 유지합니다..

DAG 루트에서 `.backward()`가 호출되면 역방향 패스가 시작됩니다. 그 다음 `autograd`는:

- 각 `.grad_fn`으로 부터 기울기를 계산합니다,
- 각 텐서의 `.grad` 속성으로 누적합니다.
- 체인 룰을 사용하여 리프 Tensors까지 전파합니다.

**DAGs are dynamic in PyTorch**

  중요한 점은 그래프가 처음부터 다시 생성된다는 것이다. 각 `.backward()` 호출 후에는 autograd가 새 그래프를 채우기 시작한다. 이것이 바로 모형에 control flow statements을 사용할 수 있도록 하는 것입니다. 필요한 경우 반복할 때마다 모양, 크기 및 연산을 변경할 수 있습니다.



## Optional reading: Tensor gradients and Jacobian products

대부분의 경우 Scalar 손실 함수를 사용하며, 일부 매개 변수와 관련하여 기울기를 계산해야 합니다. 그러나 출력 값이 임의의 Tensor인 경우 PyTorch를 사용하면 실제 기울기가 아닌 **Jacobian product**을 계산할 수 있습니다.

Jacobian 행렬 자체를 계산하는 대신 Pytorch를 사용하면 주어진 입력 Vector $v=(v_1 \dots v_m)$에 대해 Jacobian product $v^T\cdot J$를 계산할 수 있습니다. 이는 $v$를 인수로 사용하여 역방향의 호출을 통해 이루어지며, $v$의 크기는 product를 계산하려는 원래 Tensor의 크기와 같아야 합니다.

In [7]:
inp = torch.eye(5, requires_grad=True)
out = (inp+1).pow(2)
out.backward(torch.ones_like(inp), retain_graph=True)
print("First call\n", inp.grad)
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nSecond call\n", inp.grad)
inp.grad.zero_()
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nCall after zeroing gradients\n", inp.grad)

First call
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
 tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])


동일한 인수를 사용하여도 두 번째로 `backward`를 호출하면 첫 번째로 `backward`를 호출했을 때 기울기 값과는 다릅니다. 이는 역전파를 수행할 때 PyTorch가 기울기를 누적하기 때문에 발생합니다. 즉, 계산된 기울기의 값이 계산 그래프의 모든 리프 노드의 `grad` 속성에 추가되며, 적절한 기울기를 계산하려면 먼저 `grad` 속성을 0으로 설정해야 합니다. 실제 훈련에서도 최적화 작업은 이를 수행하는 데 도움이 됩니다.

> **Note:** 이전에는 매개 변수 없이 `backward()` 함수를 호출했습니다. 이는 `backward(torch.tensor(1.0))`를 호출하는 것과 동일하며, 신경망 훈련 중 Loss와 같은 Scalar 값 함수에 대한 기울기를 계산하는 유용한 방법입니다.

--------------


